In [1]:
import numpy as np 
import pandas as pd
import os
import gc
import re

# feature libraries
from scipy.sparse import hstack, csr_matrix
from sklearn import preprocessing

In [2]:
data_dir = '../input/kaggle_data'
feat_dir = '../input/features'

In [25]:
used_cols = ['item_id','region','city']

train = pd.read_csv(f'{data_dir}/train.csv', index_col="item_id", usecols=used_cols)
test = pd.read_csv(f'{data_dir}/test.csv', index_col="item_id", usecols=used_cols)

train_idx = train.index
test_idx = test.index

df = pd.concat([train, test], axis=0)

del train, test
gc.collect()

91

In [26]:
df.columns[df.isna().any()].tolist()

[]

In [27]:
locations = pd.read_csv(f'{feat_dir}/city_latlons.csv')

df['location'] = df['city'] + ', ' + df['region']
df = df.reset_index().merge(locations, how="left", on='location').set_index('item_id')
df.drop(['location','region','city'], axis=1, inplace=True)

for c in ['lat', 'lon']:
    df[c] = (df[c] - np.mean(df[c]))/np.std(df[c])


In [28]:
feat_cols = list(df.columns)[0:]
feat_cols

['lon', 'lat']

In [30]:
train_geocode = df.loc[train_idx,:]
train_geocode.head()

,lon,lat
item_id,,
b912c3c6a6ad,0.267387,0.841229
2dac0150717d,-0.300510,-0.004812
ba83aefab5dc,-0.875800,-1.417264
02996f1dd2ea,-0.182883,0.577732
7c90be56d2ab,-0.612658,-1.071007


In [31]:
test_geocode = df.loc[test_idx,:]
test_geocode.head()

,lon,lat
item_id,,
6544e41a8817,-0.612658,-1.071007
65b9484d670f,0.226383,1.261921
8bab230b2ecd,1.496439,0.355862
8e348601fefc,-0.533498,-0.392650
8bd2fe400b89,-0.188943,-0.114924


In [33]:
train_geocode.to_csv(f'{feat_dir}/train_geocode.csv', index=True, header=True)
test_geocode.to_csv(f'{feat_dir}/test_geocode.csv', index=True, header=True)